In [ ]:
import pandas as pd
from itertools import chain
from collections import Counter
import urlexpander
from urllib.parse import urlparse
import re
import numpy as np
import requests
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math
import seaborn as sns

from urllib.request import urlopen

In [ ]:
fb=pd.read_json("flashback_info_df.json") #path to the file
fb

In [ ]:
#clean strings that contain users' messages 

fb.quote = fb.quote.apply (lambda x: [x["quote"]] if len(x)>0 else [""])
fb.quote = fb.quote.apply (lambda x: x[0] if type (x[0]) == list else x)
fb.quote = fb.quote.apply (lambda x: [name.strip() for name in x])

fb.url = fb.url.apply (lambda x: [x["url"]] if len(x)>0 else [""])
fb.url = fb.url.apply (lambda x: x[0] if type (x[0]) == list else x)

fb.quotetext = fb.quotetext.apply (lambda x: [x["quotetext"]] if len(x)>0 else [""])
fb.quotetext = fb.quotetext.apply (lambda x: [""] if x == ["character(0)"] else x)
fb.quotetext = fb.quotetext.apply (lambda x: x[0] if type (x[0]) == list else x)

fb.quoteid = fb.quoteid.apply (lambda x: [x["quoteid"]] if len(x)>0 else [""])
fb.quoteid = fb.quoteid.apply (lambda x: [""] if x == ["character(0)"] else x)
fb.quoteid = fb.quoteid.apply (lambda x: x[0] if type (x[0]) == list else x)
fb.quoteid = fb.quoteid.apply (lambda x: [name.strip() for name in x])


In [ ]:
#Expand short links

bitly = fb.url.apply(lambda x: [re.findall('\S+bit\.ly.*', l)for l in x])
bitly = bitly.apply(lambda x: sum (x, []))

links=[]
for bit in bitly:
    l=[]
    for b in bit:
        try:
            exp_link = requests.get(b)
            l.append(exp_link.url)
            print(exp_link.url)
        except:
            pass
        
    links.append(l) 
    
for i in range(0, len(links)):
    
    if links[i]:
        for j in range(0, len(links[i])):
        
            fb.url[i][j]=links[i][j]    


In [ ]:
#Shorten URLs

shorturl = fb.url.apply (lambda x: [urlparse(url).hostname for url in x])
fb = fb.assign (shorturl = pd.Series(shorturl))

fb.shorturl = fb.shorturl.apply(lambda x: [re.sub('www\.', '', string or '') for string in x])

In [ ]:
#Identify the most popular media resourced and save the list for further labelling
                          
sources= [[x for x in line if x  != ''] for line in fb.shorturl]

sources_count = Counter(x for xs in sources for x in set(xs))
sources_sorted = sources_count.most_common()

source_df = pd.DataFrame.from_records(sources_sorted, columns =['name', 'freq'])

sm_list = ["youtube.com", "twitter.com", "reddit.com", "instagram.com", "facebook.com", "tiktok.com"]

source_df = source_df[~source_df['name'].isin(sm_list)]

source_df.to_excel ("urls_freq.xlsx")


In [ ]:
#import labelled media list and assign categories to each of the links

url_df_final = pd.read_excel("media_list_labelling.xlsx")
url_df_final = url_df_final.drop (columns = "Unnamed: 0")
url_df_final


media_types = []
for j in range (0, len(fb)):
    line_types = []
    if len (fb.loc[j, 'shorturl'])==0:
            
        a = 'na'
        line_types.append(a)
    else:
            
        a=[element.replace(element, url_df_final.loc[url_df_final.name == element, 'final_label'].iloc[0])
                                        if element in url_df_final.name.unique() else 'na' 
                                        for element in fb.loc[j, 'shorturl']]
        
        line_types.append(a)
        
    media_types.append(a)
     
fb = fb.assign(media_type = pd.Series(media_types))       

In [ ]:
#calculate the number of links per category in each message

num_am = fb.media_type.apply(lambda row: row.count('AM'))
fb = fb.assign (num_am = num_am)

num_mm = fb.media_type.apply(lambda row: row.count('MM'))
fb = fb.assign (num_mm = num_mm)

num_all = fb.shorturl.apply (lambda x: len (x) if x[0] else 0) 
fb = fb.assign (num_all = num_all)

In [ ]:
#save
fb.to_json("flashback_info_df.json")